In [1]:
import time, os, re
import PersianCal
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
link = 'https://www.tgju.org/profile/price_dollar_rl/history'

In [3]:
def page_changer():
    next_page_xpath = '''//*[@id="DataTables_Table_0_next"]'''
    next_page = driver.find_element_by_xpath(next_page_xpath)
    next_page.click()

In [4]:
def value_fixer(val):
    if '.' in val:
        result = float(val.replace(",", ""))
    else:
        result = int(val.replace(",", ""))
    return result

In [5]:
driver = webdriver.Chrome('chromedriver.exe')
driver.maximize_window()
driver.get(link)
time.sleep(5)

In [6]:
Open = []
Low = []
High = []
Close = []
date = []
per_date = []

In [7]:
def scraper():
    time.sleep(1)
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'lxml')
    source_data = soup.find("tbody").find_all("td")
    data_size = []

    for i in range(0, len(source_data)+1, 8):
        data_size.append(i)
        
    data = []
    for i in range(0, len(data_size)-1):
        data.append(source_data[data_size[i]:data_size[i+1]])
        
    for i in range(len(data)):
        Open.append(value_fixer(data[i][0].text))
        Low.append(value_fixer(data[i][1].text))
        High.append(value_fixer(data[i][2].text))
        Close.append(value_fixer(data[i][3].text))
        date.append(data[i][-2].text)
        per_date.append(data[-1][0].text)

In [16]:
start_time = time.time()

for i in range(87):
    scraper()
    page_changer()
    
print("--- {} seconds ---".format((time.time() - start_time)))

In [33]:
dF = {"Open": Open,
     "High": High,
     "Low": Low,
     "Close": Close,
     "Date": date,
     "Persian Date": per_date}

df = pd.DataFrame(dF)
df.Date = pd.to_datetime(df["Date"])

In [35]:
# import plotly.graph_objects as go

# fig = go.Figure(data=[go.Candlestick(x=df['Date'],
#                 open=df['Open'],
#                 high=df['High'],
#                 low=df['Low'],
#                 close=df['Close'])])

# fig.show()

In [40]:
# df.set_index("Date", inplace = True)
df.to_csv("D:\Stocks\Dollar\dollar.csv")